https://vgpena.github.io/classifying-tweets-with-keras-and-tensorflow/

En el anterior enlace, tenéis un ejemplo sobre cómo, a partir de tweets con un label específico (un sentimiento, positivo o negativo): 

1. Genera un conjunto de entrenamiento. El conjunto de entrenamiento es formado a partir de tweets completos pasados a un array con un tamaño específico.
2. Ese array (X_train de tamaño N) tiene un label que representa el sentimiento (y_train)
3. Como todas las frases tienen un tamaño N, la entrada de la red neuronal será de tamaño N y la salida de la red será de tamaño 2 usando activación softmax(porque hay dos clases).

Se pide: 

- Realizar un clasificador de reviews para el dataset de IMDB de la carpeta data_exercise/

**Cuando usa la importación "keras.x", reemplázalo por "tensorflow.keras.x"**

In [1]:
# Your code
"""
De
from keras.preprocessing.text import Tokenizer
Usa
from tensorflow.keras.preprocessing.text import Tokenizer
"""
print()

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tensorflow import keras
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

In [2]:
training = pd.read_csv("data_exercise/IMDB Dataset.csv")

In [ ]:
X_train = training["review"]
y_train = training["sentiment"]

In [ ]:
y_train.value_counts()

In [ ]:
# se asigna un valor numérico a las categorías de la variable y: sentiment (positivo:1, negativo:0)

y_train = y_train.apply(lambda x: 1 if x == "positive" else 0)


In [ ]:
# Se pasa a array la variable y_train

y_train = np.asarray(y_train)

In [ ]:
max_words = 3000
tokenizer = Tokenizer(num_words=max_words)  # Se divide el texto en 3000 palabras
tokenizer.fit_on_texts(X_train) # Se aplica la tokenización al texto de entrada, de X_train

In [ ]:

dictionary = tokenizer.word_index

with open('dictionary.json', 'w') as dictionary_file: 
    json.dump(dictionary, dictionary_file)

In [ ]:
def convert_text_to_index_array(text): # pasar las palabras del texto a un array
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

In [ ]:
# Se crea una lista con los índices de todas las palabras del texto

allWordIndices = []

for text in X_train:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

In [ ]:

allWordIndices = np.asarray(allWordIndices) # Se pasa a array la lista de índices de las palabras del texto

train_x = tokenizer.sequences_to_matrix(allWordIndices, mode='binary') # Se pasa una matriz binaria el array de índices de las palabras del texto

In [ ]:
# Se pasa categórica la variable y_train

train_y = keras.utils.to_categorical(y_train, 2)

In [ ]:
# Se establece el modelo de red neuronal 

model = Sequential()
model.add(Dense(512, input_shape=(max_words,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
# Se añaden los parámetros de compilación

model.compile(loss='categorical_crossentropy',
  optimizer='adam',
  metrics= ["CategoricalAccuracy"])

In [ ]:
# Se fija el modelo

hist_model = model.fit(train_x, train_y,
  batch_size=32,
  epochs=5,
  verbose=1,
  validation_split=0.1,
  shuffle=True)

In [ ]:
# Se obtiene la gráfica de la pérdida que se obtiene con el modelo (loss) en cada época, para los datos de entrenamiento y validación

plt.plot(hist_model.history['loss'], label=' training data')
plt.plot(hist_model.history['val_loss'], label='validation data)')
plt.title('Loss for Review sentiment Classification')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Se obtiene la gráfica de la precisión que se obtiene con el modelo (accuracy) en cada época, para los datos de entrenamiento y validación

plt.plot(hist_model.history['categorical_accuracy'], label='training data')
plt.plot(hist_model.history['val_categorical_accuracy'], label='validation data')
plt.title('CategoricalAccuracy for Text Classification')
plt.ylabel('CategoricalAccuracy value')
plt.xlabel('No. epoch')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Se pasa el modelo a json y se archiva

model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')